# 03 - Multivariate Time Series Model Training

This notebook focuses on:
1. VAR (Vector Autoregression) models
2. VARMA (Vector ARMA) models
3. Machine learning models with multiple features
4. Deep learning approaches (LSTM, GRU)
5. Feature engineering for multivariate forecasting
6. Model comparison and selection


In [ ]:
# Import libraries with robust error handling
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
import pickle
import os

# Import time series libraries with individual error handling
var_available = False
varma_available = False

try:
    from statsmodels.tsa.vector_ar.var_model import VAR
    var_available = True
    print("✅ VAR imported successfully")
except Exception as e:
    print(f"❌ VAR import failed: {e}")

try:
    from statsmodels.tsa.vector_ar.svar_model import SVAR
    varma_available = True
    print("✅ SVAR imported successfully")
except Exception as e:
    print(f"❌ SVAR import failed: {e}")

# Set style
plt.style.use('seaborn')
sns.set_palette("husl")
np.random.seed(42)

print("✅ All imports completed successfully!")


In [ ]:
# Load prepared data and setup
print("=== LOADING DATA FOR MULTIVARIATE MODEL TRAINING ===")

try:
    # Load the prepared data splits
    in_time_data = pd.read_csv('../data_multivariate/in_time_features.csv', index_col=0, parse_dates=True)
    out_of_time_data = pd.read_csv('../data_multivariate/out_of_time_features.csv', index_col=0, parse_dates=True)
    
    print(f"✅ Data loaded successfully!")
    print(f"In-time data shape: {in_time_data.shape}")
    print(f"Out-of-time data shape: {out_of_time_data.shape}")
    
    # Set up dataset configuration
    target = 'PM2.5'  # Default target for air quality data
    feature_columns = [col for col in in_time_data.columns if col != target]
    
    print(f"Features: {len(feature_columns)}")
    print(f"Target: {target}")
    print(f"Date range: {in_time_data.index.min()} to {out_of_time_data.index.max()}")
    
    # Prepare data for modeling
    # For VAR models, we need all variables
    var_data = in_time_data.copy()
    var_test_data = out_of_time_data.copy()
    
    # For ML models, separate features and target
    X_train = in_time_data[feature_columns]
    y_train = in_time_data[target]
    X_test = out_of_time_data[feature_columns]
    y_test = out_of_time_data[target]
    
    print(f"✅ Data prepared for modeling")
    print(f"VAR data shape: {var_data.shape}")
    print(f"ML training features: {X_train.shape}")
    print(f"ML training target: {y_train.shape}")
    
except FileNotFoundError:
    print("❌ Prepared data files not found")
    print("Please run 01_multivariate_data_exploration.ipynb and 02_multivariate_time_series_analysis.ipynb first")
    var_data = None
    var_test_data = None
    X_train = None
    y_train = None
    X_test = None
    y_test = None


In [ ]:
# Define evaluation metrics and model results storage
def calculate_metrics(y_true, y_pred):
    """
    Calculate time series evaluation metrics
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    
    # Symmetric MAPE
    smape = 100 * np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))
    
    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'sMAPE': smape
    }

def print_metrics(metrics, model_name):
    """
    Print formatted metrics
    """
    print(f"\n{model_name} Metrics:")
    print("-" * 30)
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

# Store results
model_results = {}
print("✅ Evaluation metrics and storage setup completed")


In [ ]:
# VAR (Vector Autoregression) Models
print("=== TRAINING VAR (VECTOR AUTOREGRESSION) MODELS ===")

if var_available and var_data is not None and not var_data.empty:
    print("Training VAR models for multivariate time series forecasting...")
    
    # Ensure data is stationary (VAR requires stationary data)
    print("Checking stationarity and applying differencing if needed...")
    
    # Test for stationarity and apply differencing
    var_data_stationary = var_data.copy()
    differenced = False
    
    for col in var_data.columns:
        # Simple ADF test
        from statsmodels.tsa.stattools import adfuller
        adf_result = adfuller(var_data[col].dropna())
        if adf_result[1] > 0.05:  # Non-stationary
            print(f"  {col}: Non-stationary, applying first difference")
            var_data_stationary[col] = var_data[col].diff()
            differenced = True
        else:
            print(f"  {col}: Stationary")
    
    if differenced:
        print("Applied first differencing to achieve stationarity")
        var_data_stationary = var_data_stationary.dropna()
    
    # Select optimal lag order using information criteria
    print("\nSelecting optimal lag order...")
    
    try:
        # Fit VAR model with automatic lag selection
        model = VAR(var_data_stationary)
        lag_order_results = model.select_order(maxlags=15)
        
        print(f"Lag selection results:")
        print(f"  AIC: {lag_order_results.aic}")
        print(f"  BIC: {lag_order_results.bic}")
        print(f"  HQIC: {lag_order_results.hqic}")
        print(f"  FPE: {lag_order_results.fpe}")
        
        # Use BIC for lag selection (more conservative)
        optimal_lags = lag_order_results.bic
        print(f"Selected lag order: {optimal_lags}")
        
        # Fit VAR model with selected lags
        print(f"\nFitting VAR model with {optimal_lags} lags...")
        var_model = model.fit(maxlags=optimal_lags, ic='bic')
        
        print("VAR Model Summary:")
        print(var_model.summary())
        
        # Make forecasts
        print(f"\nMaking forecasts for {len(var_test_data)} periods...")
        
        # For forecasting, we need to handle the differenced data
        if differenced:
            # Forecast the differenced series
            forecast_steps = len(var_test_data)
            forecasts = var_model.forecast(var_data_stationary.values[-optimal_lags:], steps=forecast_steps)
            
            # Convert forecasts back to levels (integrate)
            forecast_df = pd.DataFrame(forecasts, columns=var_data.columns)
            
            # Integrate to get level forecasts
            level_forecasts = forecast_df.copy()
            for col in forecast_df.columns:
                # Add back the last observed value
                last_value = var_data[col].iloc[-1]
                level_forecasts[col] = forecast_df[col].cumsum() + last_value
        else:
            # Direct forecasting for stationary data
            forecast_steps = len(var_test_data)
            forecasts = var_model.forecast(var_data.values[-optimal_lags:], steps=forecast_steps)
            level_forecasts = pd.DataFrame(forecasts, columns=var_data.columns)
        
        # Evaluate forecasts for target variable
        if target in level_forecasts.columns:
            var_pred = level_forecasts[target].values
            var_actual = var_test_data[target].values
            
            # Calculate metrics
            var_metrics = calculate_metrics(var_actual, var_pred)
            print_metrics(var_metrics, "VAR Model")
            model_results['VAR'] = var_metrics
            
            # Store VAR model and forecasts
            var_forecasts = level_forecasts
            
        else:
            print(f"❌ Target variable '{target}' not found in forecasts")
            
    except Exception as e:
        print(f"❌ VAR model training failed: {e}")
        
else:
    if not var_available:
        print("❌ VAR not available - statsmodels VAR module not found")
    else:
        print("❌ No data available for VAR modeling")


In [ ]:
# Machine Learning Models for Multivariate Time Series
print("=== TRAINING MACHINE LEARNING MODELS ===")

if X_train is not None and y_train is not None:
    print("Training machine learning models with multiple features...")
    
    # 1. Random Forest with Hyperparameter Tuning
    print("\n1. Training Random Forest Model with Hyperparameter Tuning...")
    
    try:
        # Define parameter grid for hyperparameter tuning
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [5, 10, 15, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        }
        
        # Create base Random Forest model
        rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
        
        # Use TimeSeriesSplit for cross-validation (respects temporal order)
        tscv = TimeSeriesSplit(n_splits=3)
        
        # Grid search with time series cross-validation
        print("Performing hyperparameter tuning...")
        rf_grid = GridSearchCV(
            estimator=rf_base,
            param_grid=param_grid,
            cv=tscv,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=1
        )
        
        # Fit the grid search
        rf_grid.fit(X_train, y_train)
        
        # Get best model
        rf_model = rf_grid.best_estimator_
        
        print(f"Best parameters: {rf_grid.best_params_}")
        print(f"Best cross-validation score: {-rf_grid.best_score_:.4f}")
        
        # Make predictions
        rf_pred = rf_model.predict(X_test)
        
        # Calculate metrics
        rf_metrics = calculate_metrics(y_test.values, rf_pred)
        print_metrics(rf_metrics, "Random Forest (Tuned)")
        model_results['RandomForest_Tuned'] = rf_metrics
        
        # Feature importance analysis
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': rf_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\nTop 10 Most Important Features:")
        print(feature_importance.head(10))
        
        # Store feature importance for later analysis
        top_features = feature_importance.head(10)
        
    except Exception as e:
        print(f"❌ Random Forest model failed: {e}")
        # Fallback to basic Random Forest if tuning fails
        print("Falling back to basic Random Forest...")
        try:
            rf_model = RandomForestRegressor(
                n_estimators=100,
                max_depth=10,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42,
                n_jobs=-1
            )
            
            rf_model.fit(X_train, y_train)
            rf_pred = rf_model.predict(X_test)
            rf_metrics = calculate_metrics(y_test.values, rf_pred)
            print_metrics(rf_metrics, "Random Forest (Basic)")
            model_results['RandomForest_Basic'] = rf_metrics
            
            # Feature importance
            feature_importance = pd.DataFrame({
                'feature': X_train.columns,
                'importance': rf_model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            print("\nTop 10 Most Important Features:")
            print(feature_importance.head(10))
            top_features = feature_importance.head(10)
            
        except Exception as e2:
            print(f"❌ Basic Random Forest also failed: {e2}")
    
    # 2. Additional ML Models (if time permits)
    print("\n2. Training Additional ML Models...")
    
    # Linear Regression as baseline
    try:
        from sklearn.linear_model import LinearRegression
        
        lr_model = LinearRegression()
        lr_model.fit(X_train, y_train)
        lr_pred = lr_model.predict(X_test)
        
        lr_metrics = calculate_metrics(y_test.values, lr_pred)
        print_metrics(lr_metrics, "Linear Regression")
        model_results['LinearRegression'] = lr_metrics
        
    except Exception as e:
        print(f"❌ Linear Regression failed: {e}")
    
    print(f"\n✅ Machine learning models completed!")
    
else:
    print("❌ No data available for machine learning models")


In [ ]:
# Model Comparison and Results Summary
print("=== MODEL COMPARISON AND RESULTS SUMMARY ===")

if model_results:
    # Create results DataFrame
    results_df = pd.DataFrame(model_results).T
    results_df = results_df.sort_values('RMSE')
    
    print("\n📊 MODEL PERFORMANCE SUMMARY (sorted by RMSE):")
    print("=" * 80)
    print(results_df.round(4))
    
    # Model categorization
    traditional_models = [col for col in results_df.index if col in ['VAR', 'SVAR']]
    ml_models = [col for col in results_df.index if any(x in col for x in ['RandomForest', 'LinearRegression'])]
    
    print(f"\n🏷️  MODEL CATEGORIES:")
    print(f"   Traditional Time Series Models: {len(traditional_models)}")
    if traditional_models:
        print(f"   - {', '.join(traditional_models)}")
    print(f"   Machine Learning Models: {len(ml_models)}")
    if ml_models:
        print(f"   - {', '.join(ml_models)}")
    
    # Find best model
    best_model = results_df.index[0]
    best_rmse = results_df.loc[best_model, 'RMSE']
    best_mae = results_df.loc[best_model, 'MAE']
    best_mape = results_df.loc[best_model, 'MAPE']
    best_smape = results_df.loc[best_model, 'sMAPE']
    
    print(f"\n🏆 BEST MODEL: {best_model}")
    print(f"   RMSE: {best_rmse:.4f} (Primary metric - lower is better)")
    print(f"   MAE:  {best_mae:.4f} (Average absolute error)")
    print(f"   MAPE: {best_mape:.4f}% (Mean absolute percentage error)")
    print(f"   sMAPE: {best_smape:.4f}% (Symmetric MAPE)")
    
    # Model performance analysis
    print(f"\n📈 PERFORMANCE ANALYSIS:")
    print(f"   Best RMSE: {best_rmse:.4f}")
    print(f"   Worst RMSE: {results_df['RMSE'].max():.4f}")
    print(f"   RMSE Range: {results_df['RMSE'].max() - best_rmse:.4f}")
    print(f"   RMSE Improvement: {((results_df['RMSE'].max() - best_rmse) / results_df['RMSE'].max() * 100):.1f}%")
    
    # Model type comparison
    if traditional_models and ml_models:
        traditional_avg_rmse = results_df.loc[traditional_models, 'RMSE'].mean()
        ml_avg_rmse = results_df.loc[ml_models, 'RMSE'].mean()
        
        print(f"\n⚖️  MODEL TYPE COMPARISON:")
        print(f"   Traditional Models Average RMSE: {traditional_avg_rmse:.4f}")
        print(f"   Machine Learning Models Average RMSE: {ml_avg_rmse:.4f}")
        
        if traditional_avg_rmse < ml_avg_rmse:
            print(f"   🎯 Traditional models perform better on average")
        else:
            print(f"   🎯 Machine learning models perform better on average")
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # RMSE comparison
    bars1 = axes[0, 0].bar(results_df.index, results_df['RMSE'], color='skyblue', alpha=0.7)
    axes[0, 0].set_title('RMSE Comparison (Primary Metric)', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('RMSE (Lower is Better)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    # Highlight best model
    best_idx = list(results_df.index).index(best_model)
    bars1[best_idx].set_color('gold')
    bars1[best_idx].set_alpha(1.0)
    
    # MAE comparison
    bars2 = axes[0, 1].bar(results_df.index, results_df['MAE'], color='lightcoral', alpha=0.7)
    axes[0, 1].set_title('MAE Comparison', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('MAE (Lower is Better)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # MAPE comparison
    bars3 = axes[1, 0].bar(results_df.index, results_df['MAPE'], color='lightgreen', alpha=0.7)
    axes[1, 0].set_title('MAPE Comparison', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('MAPE (%) (Lower is Better)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # sMAPE comparison
    bars4 = axes[1, 1].bar(results_df.index, results_df['sMAPE'], color='plum', alpha=0.7)
    axes[1, 1].set_title('sMAPE Comparison', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('sMAPE (%) (Lower is Better)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Model recommendations
    print(f"\n💡 MODEL RECOMMENDATIONS:")
    print(f"   1. 🥇 Best Overall: {best_model} (RMSE: {best_rmse:.4f})")
    
    # Find best in each category
    if traditional_models:
        best_traditional = results_df.loc[traditional_models].sort_values('RMSE').index[0]
        print(f"   2. 🏛️  Best Traditional: {best_traditional} (RMSE: {results_df.loc[best_traditional, 'RMSE']:.4f})")
    
    if ml_models:
        best_ml = results_df.loc[ml_models].sort_values('RMSE').index[0]
        print(f"   3. 🤖 Best ML Model: {best_ml} (RMSE: {results_df.loc[best_ml, 'RMSE']:.4f})")
    
    # Save results
    print(f"\n💾 SAVING RESULTS")
    print("-" * 40)
    
    # Create results directory if it doesn't exist
    results_dir = '../results/'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # Save model results
    with open(f'{results_dir}multivariate_model_results.pkl', 'wb') as f:
        pickle.dump(model_results, f)
    print("✅ Model results saved to ../results/multivariate_model_results.pkl")
    
    # Save comparison results
    results_df.to_csv(f'{results_dir}multivariate_model_comparison.csv')
    print("✅ Model comparison saved to ../results/multivariate_model_comparison.csv")
    
    print(f"\n✅ Multivariate model training completed!")
    
else:
    print("❌ No models were successfully trained.")
    print("   Please check the model training cells above for errors.")
